In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pkg_resources import ContextualVersionConflict

# from rich.console import Console

import requests
import json

import time
from sqlalchemy import delete
from tqdm import tqdm

# console = Console()

sample_transport=RequestsHTTPTransport(
    url='http://localhost:4001/graphql',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    #execute_timeout=20,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

session = requests.Session()
session.auth = ('admin', "secret")

hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
auth = session.post(hostname)
response = json.loads(session.get(hostname).content)

#tilt = json.loads(requests.get('http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt').content)
#print("Fetch Successful!", response)


json_file = open('/Users/johannes/Desktop/TU/playground/tilt')
tilt = json.load(json_file)

In [71]:
for sub in tilt[1]: 
    print(tilt[1][sub])

{'$oid': '62559206e7dfaf542de42c5d'}
{'_id': 'e031f5ee-ee8f-41a6-924c-d071dda1ea89', 'name': 'Adobe', 'created': '2021-07-30T11:44:55.119684', 'modified': '2021-07-30T11:44:55.119704', 'version': 1, 'language': 'de', 'status': 'active', 'url': 'https://www.adobe.com/de/privacy/policy.html', '_hash': 'f9987454558443041ee3214688187950a5df17d52a55473e3c8d689c9a923acf'}
{'name': None, 'country': None, 'address': None, 'division': None, 'representative': {'name': None, 'email': None, 'phone': None}}
{'email': None, 'phone': None, 'country': None, 'name': None, 'address': None}
[{'_id': [], 'category': [], 'purposes': [], 'legalBases': [], 'legitimateInterests': [], 'recipients': [], 'recipientsOnlyCategory': [], 'storage': [], 'nonDisclosure': []}]
[{'country': [], 'adequacyDecision': [], 'appropriateGuarantees': [], 'presenceOfEnforceableRightsAndEffectiveRemedies': [], 'standardDataProtectionClause': []}]
{'available': None, 'description': None, 'url': None, 'email': None, 'identification

In [126]:
def create_node(parent):

    """
    func:   create_node
            Recursively goes through JSON and builds a mutation to generate the neo4j graph
            Base case is when an entry is a string
    """

    #mutation string
    label_string = ""

    #what should be printed in return statement
    #second_string = ""

    #iterate through elements of dictionary or list
    for i, key in enumerate(parent):

        if key[0] == "$":
            return ""

        # if key is a list
        if isinstance(parent[key], list):

            #create new node
            #label_string += f'\n{key}:\n{{create:[{{{create_node(key)}}}]}}'
            label_string += f'\n{key}:\n{{create: A LIST}}'
            #second_string += f"\n{parent}s"

        # if the element is a dictionary
        if isinstance(parent[key], dict): 
            
            #create the next node for this dict
            label_string += f'\n{key}:\n{{create:[{{{create_node(parent[key])}}}]}}'
            #second_string += f"\n{key}s" 

        else: # base case
            
            if i < len(parent)-1:

                label_string += f'\n{key}: "{parent[key]}", '
                #second_string += f"{key} "
            else:
                label_string += f'\n{key}: "{parent[key]}"'
                #second_string += f"{key}" + "}"
     
    return label_string

    

In [127]:
print(f"mutation {create_node(tilt[1])}")

mutation 
_id:
{create:[{}]}
meta:
{create:[{
_id: "e031f5ee-ee8f-41a6-924c-d071dda1ea89", 
name: "Adobe", 
created: "2021-07-30T11:44:55.119684", 
modified: "2021-07-30T11:44:55.119704", 
version: "1", 
language: "de", 
status: "active", 
url: "https://www.adobe.com/de/privacy/policy.html", 
_hash: "f9987454558443041ee3214688187950a5df17d52a55473e3c8d689c9a923acf"}]}
controller:
{create:[{
name: "None", 
country: "None", 
address: "None", 
division: "None", 
representative:
{create:[{
name: "None", 
email: "None", 
phone: "None"}]}}]}
dataProtectionOfficer:
{create:[{
email: "None", 
phone: "None", 
country: "None", 
name: "None", 
address: "None"}]}
dataDisclosed:
{create: A LIST}
dataDisclosed: "[{'_id': [], 'category': [], 'purposes': [], 'legalBases': [], 'legitimateInterests': [], 'recipients': [], 'recipientsOnlyCategory': [], 'storage': [], 'nonDisclosure': []}]", 
thirdCountryTransfers:
{create: A LIST}
thirdCountryTransfers: "[{'country': [], 'adequacyDecision': [], 'appropri

In [111]:
type(tilt[1]['dataDisclosed'])

list